In [1]:
import cv2
import numpy as np
import os
import copy

In [2]:
# Input and output folder paths
# Output folder will be createad if it doesn't exist
input_path = 'images'
output_path = 'output_images'
results_path = 'results'

# Create the output folder if it does not exist
os.makedirs(output_path, exist_ok=True)

# Create the results folder if it does not exist
os.makedirs(results_path, exist_ok=True)

# Define maximum dimensions for the resized image
MAX_WIDTH = 1400  # Maximum width of the displayed image
MAX_HEIGHT = 1200  # Maximum height of the displayed image

# Matrix for the structure of the board
board = [
    ['3x', '*', '*', '*', '*', '*', '3x', '3x', '*', '*', '*', '*', '*', '3x'],
    ['*', '2x', '*', '*', '/', '*', '*', '*', '*', '/', '*', '*', '2x', '*'],
    ['*', '*', '2x', '*', '*', '-', '*', '*', '-', '*', '*', '2x', '*', '*'],
    ['*', '*', '*', '2x', '*', '*', '+', 'x', '*', '*', '2x', '*', '*', '*'],
    ['*', '/', '*', '*', '2x', '*', 'x', '+', '*', '2x', '*', '*', '/', '*'],
    ['*', '*', '-', '*', '*', '*', '*', '*', '*', '*', '*', '-', '*', '*'],
    ['3x', '*', '*', 'x', '+', '*', '1', '2', '*', 'x', '+', '*', '*', '3x'],
    ['3x', '*', '*', '+', 'x', '*', '3', '4', '*', '+', 'x', '*', '*', '3x'],
    ['*', '*', '-', '*', '*', '*', '*', '*', '*', '*', '*', '-', '*', '*'],
    ['*', '/', '*', '*', '2x', '*', '+', 'x', '*', '2x', '*', '*', '/', '*'],
    ['*', '*', '*', '2x', '*', '*', 'x', '+', '*', '*', '2x', '*', '*', '*'],
    ['*', '*', '2x', '*', '*', '-', '*', '*', '-', '*', '*', '2x', '*', '*'],
    ['*', '2x', '*', '*', '/', '*', '*', '*', '*', '/', '*', '*', '2x', '*'],
    ['3x', '*', '*', '*', '*', '*', '3x', '3x', '*', '*', '*', '*', '*', '3x'],
]

# Matrix for placing the pieces as we go along
board_with_pieces = [
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, 1, 2, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, 3, 4, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
]

# Values for extracting the grid
min_hue = 13
max_hue = 104
min_sat = 0
max_sat = 83
min_val = 0
max_val = 255
lower_bound = np.array([min_hue, min_sat, min_val])
upper_bound = np.array([max_hue, max_sat, max_val])

# Values for extracting the first piece
min_hue_p = 0
max_hue_p = 87
min_sat_p = 0
max_sat_p = 86
min_val_p = 180
max_val_p = 255 
lower_bound_p = np.array([min_hue_p, min_sat_p, min_val_p])
upper_bound_p = np.array([max_hue_p, max_sat_p, max_val_p])

# For retaining the pieces already placed on the board
existing_pieces = []
# For memorizing the templates after processing
templates_proc = []
# Dictionary for retaining the pieces and the cells in which they are placed
pieces = []
# Memorize the score for every placed piece
scores_per_piece = None
# For memorizing the processed images
batch_images = {}

In [3]:
def resize_image(image, max_width, max_height):
    #Resize image to fit within max dimensions while maintaining aspect ratio
    height, width = image.shape[:2]
    aspect_ratio = width / height

    if width > max_width or height > max_height:
        if aspect_ratio > 1:
            # Landscape orientation - scale by width
            new_width = max_width
            new_height = int(max_width / aspect_ratio)
        else:
            # Portrait orientation - scale by height
            new_height = max_height
            new_width = int(max_height * aspect_ratio)

        # Resize the image
        image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)

    return image

In [4]:
def apply_mask(image, lower_bound, upper_bound):
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Binary mask for the thresholded hsv values
    mask = cv2.inRange(hsv_img, lower_bound, upper_bound)

    # Clean mask up
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Detect contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the largest contour
    largest_contour = max(contours, key=cv2.contourArea)

    # Get the bounding box
    x, y, w, h = cv2.boundingRect(largest_contour)

    # Create a mask for the bounding box
    bounding_box_mask = np.zeros_like(mask)
    cv2.rectangle(bounding_box_mask, (x, y), (x + w, y + h), (255), thickness=cv2.FILLED)

    img_copy = image.copy()
    #masked_image = cv2.bitwise_and(img_copy, img_copy, mask=bounding_box_mask)
    #cropped_image = image[y : y + h, x : x + w]

    # Dimensions for perspective wrap
    width_r = 896
    height_r = 896

    puzzle = np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]], dtype = "float32")
    destination_of_puzzle = np.array([[0, 0], [width_r, 0], [width_r, height_r], [0, height_r]], dtype = "float32")

    M = cv2.getPerspectiveTransform(puzzle, destination_of_puzzle)

    result = cv2.warpPerspective(img_copy, M, (width_r, height_r))
    return result

In [5]:
def detect_piece(grid_image, lower_bound, upper_bound):
    hsv_img = cv2.cvtColor(grid_image, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv_img, lower_bound, upper_bound)

    # Clean mask up
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find mask contour
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)

    # Find the center of the piece
    M = cv2.moments(largest_contour)
    if M["m00"] == 0:
        print("Contour center could not be calculated.")
        return None
    center_x = int(M["m10"] / M["m00"])
    center_y = int(M["m01"] / M["m00"])

    # Calculate grid cell dimensions
    height, width = grid_image.shape[:2]
    cell_width = width // 14
    cell_height = height // 14

    # Determine the row and column of the cell containing the piece
    col = center_x // cell_width
    row = center_y // cell_height

    return row, col

In [6]:
def process_templates(input_folder, output_folder, template_size=(64, 64)):
    
    # Ensure existance of output folder
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through all templates
    for filename in os.listdir(input_folder):
        # Check if the file is an image
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Load the template
            file_path = os.path.join(input_folder, filename)
            template = cv2.imread(file_path)

            # Grayscale conversion
            gray_template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

            # Noise reduction with median blur
            denoised_template = cv2.medianBlur(gray_template, 3)

            # Thresholding
            _, binary_template = cv2.threshold(denoised_template, 127, 255, cv2.THRESH_BINARY)

            """ # Morphological operation
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
            cleaned_templates = cv2.morphologyEx(binary_template, cv2.MORPH_CLOSE, kernel) """

            binary_template = cv2.bitwise_not(binary_template)

            # Resize to standard dimension
            resized_template = cv2.resize(binary_template, template_size, interpolation=cv2.INTER_AREA)

            templates_proc.append((filename.replace(".jpg", ""), resized_template))

            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, resized_template)

    print(f"Templates saved to {output_folder}")

In [7]:
def detect_next_piece_hsv(curr_image, prev_image):
    # Comnvert images to hsv space
    hsv_curr = cv2.cvtColor(curr_image, cv2.COLOR_BGR2HSV)
    hsv_prev = cv2.cvtColor(prev_image, cv2.COLOR_BGR2HSV)

    # Calculating hsv masks for piece separation
    mask_curr = cv2.inRange(hsv_curr, lower_bound_p, upper_bound_p)
    mask_prev = cv2.inRange(hsv_prev, lower_bound_p, upper_bound_p)

    # Calculating difference between masks
    mask_diff = cv2.bitwise_xor(mask_curr, mask_prev)

    # Clean up the mask using morphological operations
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask_diff, cv2.MORPH_CLOSE, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)

    # Dilate mask to make features more pronounced
    mask = cv2.dilate(mask, kernel, iterations=1)

    """ # Display the mask
    cv2.imshow("Mask", mask)  # OpenCV window to display the mask
    cv2.waitKey(0)  # Wait for a key press to close the window
    cv2.destroyAllWindows() """

    # Find contours of the new piece
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        print("No new piece detected.")
        return None, mask

    # Assume the largest contour corresponds to the new piece
    largest_contour = max(contours, key=cv2.contourArea)

    # Convert the mask to a 3-channel BGR image
    mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    # Draw the largest contour on the mask image
    cv2.drawContours(mask_bgr, [largest_contour], -1, (0, 0, 255), 2)  # Red contour with thickness 2

    """ # Display the mask with the largest contour
    cv2.imshow("Largest Contour on Mask", mask_bgr)
    cv2.waitKey(0)
    cv2.destroyAllWindows() """

    # Find the center of the new piece
    M = cv2.moments(largest_contour)
    if M["m00"] == 0:
        print("Contour center could not be calculated.")
        return None, mask
    center_x = int(M["m10"] / M["m00"])
    center_y = int(M["m01"] / M["m00"])

    # Calculate grid cell dimensions
    height, width = curr_image.shape[:2]
    cell_width = width // 14
    cell_height = height // 14

    # Determine the row and column of the cell containing the new piece
    col = center_x // cell_width
    row = center_y // cell_height

    image = curr_image.copy()

    for r in range(14 + 1):
        cv2.line(image, (0, r * cell_height), (width, r * cell_height), (255, 0, 0), 1)

    for c in range(14 + 1):
        cv2.line(image, (c * cell_width, 0), (c * cell_width, height), (255, 0, 0), 1)

    """ cv2.circle(image, (center_x, center_y), 5, (0, 255, 0), -1)
    cv2.imshow('Grid with Centroid', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows() """

    return row, col

In [8]:
# Function to compute the minimum distance between two contours
def min_distance_between_contours(contour1, contour2):
    min_distance = float('inf')
    for point1 in contour1:
        for point2 in contour2:
            distance = np.linalg.norm(point1 - point2)  # Euclidean distance
            if distance < min_distance:
                min_distance = distance
    return min_distance

def process_piece(image):

    lower_bound_black = np.array([0, 0, 0])
    upper_bound_black = np.array([179, 255, 85])

    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    mask = cv2.GaussianBlur(image, (5, 5), 0)

    mask = cv2.inRange(mask, lower_bound_black, upper_bound_black)

    # Create the unsharp mask
    mask = cv2.addWeighted(mask, 1.5, mask, -0.5, 0)

    # Dilate mask to make features more pronounced
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=1)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    min_area_threshold = 50  # Adjust this based on your images
    max_distance_threshold = 15
    contours = [contour for contour in contours if cv2.contourArea(contour) > min_area_threshold]

    if not contours:
        print("No contours found in the image.")

    # Sort contours by area in descending order
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Ensure we have at least two contours
    if len(contours) < 2:
        #print("Warning: Less than two contours found. Using single largest contour.")
        contours_to_merge = contours
    else:
        if min_distance_between_contours(contours[0], contours[1]) <= max_distance_threshold:
            contours_to_merge = contours[:2]  # Take the two largest contours
        else:
            contours_to_merge = contours[:1]

    # Create a combined bounding box for the two largest contours
    x_min, y_min, x_max, y_max = float('inf'), float('inf'), 0, 0
    for contour in contours_to_merge:
        x, y, w, h = cv2.boundingRect(contour)
        x_min = min(x_min, x)
        y_min = min(y_min, y)
        x_max = max(x_max, x + w)
        y_max = max(y_max, y + h)

    # Extract the ROI
    roi = mask[y_min:y_max, x_min:x_max]
    
    return roi

In [9]:
def match_numbers(pieces, templates):
    for item in pieces:
        cell_image = item['piece']
        best_match = None
        best_score = -1
        best_template_name = None

        for template_name, template_image in templates:
            # Finding contours
            piece_contours, _ = cv2.findContours(cell_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            #template_image = cv2.cvtColor(template_image, cv2.COLOR_BGR2GRAY)
            template_contours, _ = cv2.findContours(template_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            min_area_threshold = 64  # Adjust this based on your images
            contours = [contour for contour in template_contours if cv2.contourArea(contour) > min_area_threshold]

            # Sort contours by area in descending order
            contours = sorted(contours, key=cv2.contourArea, reverse=True)

            # Ensure we have at least two contours
            if len(contours) < 2:
                #print("Warning: Less than two contours found. Using single largest contour.")
                contours_to_merge = contours
            else:
                if len(template_name) == 2:
                    contours_to_merge = contours[:2]  # Take the two largest contours
                else:
                    contours_to_merge = contours[:1]

            # Create a combined bounding box for the two largest contours
            x_min, y_min, x_max, y_max = float('inf'), float('inf'), 0, 0
            for contour in contours_to_merge:
                x, y, w, h = cv2.boundingRect(contour)
                x_min = min(x_min, x)
                y_min = min(y_min, y)
                x_max = max(x_max, x + w)
                y_max = max(y_max, y + h)

            # Extract the ROI
            template_cropped = template_image[y_min:y_max, x_min:x_max]

            height_c, width_c = cell_image.shape[:2]

            #template_cropped = resize_image(template_cropped, width_c, height_c)
            template_cropped = cv2.resize(template_cropped, (width_c, height_c), cv2.INTER_AREA)

            #_, template_cropped = cv2.threshold(template_cropped, 5, 255, cv2.THRESH_BINARY)

            score = cv2.matchTemplate(template_cropped, cell_image, cv2.TM_CCOEFF_NORMED)
            max_val = np.max(score)

            # Update best match
            if max_val > best_score:
                best_score = max_val
                best_match = template_cropped
                best_template_name = template_name
        
        item['number'] = int(best_template_name)

        #print(f"best match: {best_template_name} - score: {best_score}")

        """ cv2.imshow("Cell piece", cell_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows() """

In [10]:
def determine_piece_scores(pieces, batch_number):
    scores_per_piece = []

    board_with_pieces_copy = copy.deepcopy(board_with_pieces)

    # Itereate through the pieces 
    for item in pieces:
        if item['batch'] == batch_number:
            row = item['row'] - 1
            col = item['col'] - 1
            number = item['number']

            directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
            score = 0
            restriction = board[row][col]

            for dr, dc in directions:
                first_row, first_col = row + dr, col + dc
                second_row, second_col = first_row + dr, first_col + dc

                #print(f"Positions checked: {first_row}, {first_col} | {second_row}, {second_col}")

                # Ensure indices are within bounds
                if (
                    0 <= first_row < 14 and 0 <= first_col < 14 and board_with_pieces_copy[first_row][first_col] != -1 and
                    0 <= second_row < 14 and 0 <= second_col < 14 and board_with_pieces_copy[second_row][second_col] != -1
                ):
                    first_value = board_with_pieces_copy[first_row][first_col]
                    second_value = board_with_pieces_copy[second_row][second_col]

                    #print(f"Checking for number {number}, row: {row}, column: {col}")
                    #print(second_value, first_value, restriction)

                    # Check if the piece solves a valid equation
                    if restriction == '+':
                        if number == first_value + second_value:
                            score += number
                    elif restriction == '-':
                        if number == abs(first_value - second_value):
                            score += number
                    elif restriction == 'x':
                        if number == first_value * second_value:
                            score += number
                    elif restriction == '/':
                        if second_value != 0 and number == first_value // second_value:
                            score += number
                        elif first_value != 0 and number == second_value // first_value:
                            score += number
                    else:
                        # No restriction, check all operations
                        if (
                            number == first_value + second_value or
                            number == abs(first_value - second_value) or
                            number == first_value * second_value or
                            (second_value != 0 and number == first_value // second_value) or
                            (first_value != 0 and number == second_value // first_value)
                        ):
                            score += number
                        
                    #print(f"final score for checking: {score}")

            # Check for multipliers
            if restriction == '2x':
                score *= 2
            elif restriction == '3x':
                score *= 3 

            board_with_pieces_copy[row][col] = number

            scores_per_piece.append(score)

    return scores_per_piece

In [11]:
def write_final_scores(scores_per_piece, input_file, output_file):
    # Calculating the scores for each player based on the turns
    intervals = []

    with open(input_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            intervals.append(int(parts[1]))

    # Adding the last interval
    intervals.append(50)

    scores_first_player = []
    scores_second_player = []

    count = 0
    final_numbers = None

    for i in range(len(intervals) - 1):
        score = 0
        for j in range(intervals[i], intervals[i + 1]):
            score += scores_per_piece[j - 1]
            
        if count % 2 == 0:
            scores_first_player.append(score)
        else:
            scores_second_player.append(score)
        count += 1
        final_numbers = j

    for i in range(final_numbers, 50):
        if(count - 1) % 2 == 0:
            scores_first_player[len(scores_first_player) - 1] += scores_per_piece[i]
        else:
            scores_second_player[len(scores_second_player) - 1] += scores_per_piece[i]

    final_list = []
    min_len = min(len(scores_first_player), len(scores_second_player))

    # Add elements alternately
    for i in range(min_len):
        final_list.append(scores_first_player[i])
        final_list.append(scores_second_player[i])

    # Add the remaining elements from the longer list
    final_list.extend(scores_first_player[min_len:])
    final_list.extend(scores_second_player[min_len:])

    modified_lines = []
    for i in range(len(lines)):
        modified_line = f"{lines[i].strip()} {final_list[i]}"
        modified_lines.append(modified_line)

    with open(output_file, 'w') as new_file:
        for line in modified_lines:
            new_file.write(line + '\n')

In [12]:
# Process all images in the folder
for file_name in os.listdir(input_path):
    file_path = os.path.join(input_path, file_name)

    # Checking if file is an image
    if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        # Read the image
        image = cv2.imread(file_path)
        if image is None:
            print(f"Error reading image: {file_name}")
            continue

        # Resizing and applying mask
        resized_image = resize_image(image, MAX_WIDTH, MAX_HEIGHT)

        result = apply_mask(resized_image, lower_bound, upper_bound)

        batch_number = file_name.split('_')[0]
        if batch_number not in batch_images:
            batch_images[batch_number] = []
        batch_images[batch_number].append(result)

        # Save the result to the output folder
        output_file_path = os.path.join(output_path, file_name)
        cv2.imwrite(output_file_path, result)
        print(f"Processed and saved: {file_name}")

print("Batch processing complete.")

Processed and saved: 1_01.jpg
Processed and saved: 1_02.jpg
Processed and saved: 1_03.jpg
Processed and saved: 1_04.jpg
Processed and saved: 1_05.jpg
Processed and saved: 1_06.jpg
Processed and saved: 1_07.jpg
Processed and saved: 1_08.jpg
Processed and saved: 1_09.jpg
Processed and saved: 1_10.jpg
Processed and saved: 1_11.jpg
Processed and saved: 1_12.jpg
Processed and saved: 1_13.jpg
Processed and saved: 1_14.jpg
Processed and saved: 1_15.jpg
Processed and saved: 1_16.jpg
Processed and saved: 1_17.jpg
Processed and saved: 1_18.jpg
Processed and saved: 1_19.jpg
Processed and saved: 1_20.jpg
Processed and saved: 1_21.jpg
Processed and saved: 1_22.jpg
Processed and saved: 1_23.jpg
Processed and saved: 1_24.jpg
Processed and saved: 1_25.jpg
Processed and saved: 1_26.jpg
Processed and saved: 1_27.jpg
Processed and saved: 1_28.jpg
Processed and saved: 1_29.jpg
Processed and saved: 1_30.jpg
Processed and saved: 1_31.jpg
Processed and saved: 1_32.jpg
Processed and saved: 1_33.jpg
Processed 

In [13]:
for batch_number in range(1, 5):
    first_processed_img = cv2.imread('output_images/' + str(batch_number) + '_01.jpg')

    row, col = detect_piece(first_processed_img, lower_bound_p, upper_bound_p)
    height, width = first_processed_img.shape[:2]
    cell_width = width // 14
    cell_height = height // 14
    cropped_image = first_processed_img[cell_height * row : cell_height * (row + 1), cell_width * col : cell_width * (col + 1)]
    pieces.append({"batch": batch_number, "name": str(batch_number) + '_01', "row": row + 1, "col": col + 1, "piece": cropped_image, "number": None})

    prev_image = first_processed_img
    counter = 0

    for file_name in os.listdir(output_path):
        file_path = os.path.join(output_path, file_name)

        # Checking if file is an image
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')) and file_name.startswith(str(batch_number)):
            # Read the image
            image = cv2.imread(file_path)
            if image is None:
                print(f"Error reading image: {file_name}")
                continue

            """ cv2.imshow("hoe", image)
            cv2.waitKey(0)
            cv2.destroyAllWindows() """

            # Increase counter to skip over the first image of the batch
            # As we have already processed it
            if counter == 0:
                counter += 1
                continue

            row, col = detect_next_piece_hsv(image, prev_image)
            counter += 1

            # Add the piece
            height, width = image.shape[:2]
            cell_width = width // 14
            cell_height = height // 14
            cropped_image = image[cell_height * row : cell_height * (row + 1), cell_width * col : cell_width * (col + 1)]

            pieces.append({"batch": batch_number, "name": file_name.replace(".jpg", ""), "row": row + 1, "col": col + 1, "piece": cropped_image, "number": None})

            prev_image = image



In [14]:
# Processing the pieces
pieces_copy = pieces

print(len(pieces))

for item in pieces_copy:
    image = item['piece']
    
    item['piece'] = process_piece(image)

200


In [15]:
# Process the templates
process_templates('templates', 'templates_processed')

# Performing template matching to determine the number on each piece
match_numbers(pieces_copy, templates_proc)

# Creating the files for the pieces (position + number)
for item in pieces_copy:
    with open(results_path + '/' + item['name'] + '.txt', 'w') as write_file:
        contents = f"{item['row']}{chr(item['col'] + 65 - 1)} {item['number']}"
        write_file.write(contents)

# Creating the file for the scores
for batch_number in range(1, 5):
    # Calculating the scores for each placed piece
    scores_per_piece = determine_piece_scores(pieces_copy, batch_number)

    print(scores_per_piece)
    
    # Writing the final scores to the output file
    write_final_scores(scores_per_piece, 'images/' + str(batch_number) + '_turns.txt', 'results/' + str(batch_number) + '_scores.txt')

Templates saved to templates_processed
[2, 6, 7, 1, 16, 15, 14, 11, 2, 13, 50, 40, 7, 24, 5, 30, 10, 20, 12, 2, 7, 19, 36, 9, 9, 81, 4, 6, 36, 126, 2, 8, 1, 35, 210, 28, 4, 16, 10, 42, 2, 1, 3, 5, 6, 2, 4, 4, 7, 7]
[2, 3, 6, 7, 9, 7, 3, 11, 16, 0, 0, 6, 4, 20, 6, 17, 56, 27, 9, 54, 6, 27, 5, 10, 2, 6, 4, 20, 13, 14, 8, 48, 15, 10, 5, 24, 18, 10, 21, 2, 21, 4, 10, 2, 6, 4, 2, 8, 2, 10]
[3, 7, 8, 10, 2, 80, 20, 4, 6, 20, 7, 3, 20, 20, 2, 7, 3, 3, 18, 5, 4, 7, 14, 4, 63, 2, 64, 54, 5, 49, 15, 4, 20, 27, 7, 24, 34, 14, 10, 11, 70, 7, 6, 9, 13, 3, 5, 30, 80, 10]
[4, 7, 28, 35, 63, 6, 1, 1, 4, 24, 20, 2, 4, 4, 8, 8, 7, 17, 4, 19, 10, 14, 16, 80, 2, 4, 8, 1, 3, 1, 81, 3, 18, 21, 6, 60, 18, 54, 6, 10, 2, 45, 8, 10, 27, 9, 0, 3, 30, 30]
